In [ ]:
from google.colab import files
import pandas as pd

In [ ]:
uploaded = files.upload()
file_name = next(iter(uploaded))
dataframe = pd.read_excel(file_name)

dataframe.head()

Saving df_7.xlsx to df_7.xlsx


,Unnamed: 0,review,score
0,21464,"After waiting years for the sequel, this was s...",4
1,21465,"To make things simple, gameplay and graphics a...",4
2,21466,"Can't say it's a particularly enjoyable game, ...",4
3,21467,Decent gameplay and fantastic visuals along wi...,4
4,21468,Severely lacking in the story department to th...,4


In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [ ]:
def split_text_into_chunks(text, max_length=1024):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if sum(len(w) for w in current_chunk) + len(current_chunk) + len(word) <= max_length:
            current_chunk.append(word)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def analyze_long_review(review):
    chunks = split_text_into_chunks(review)
    sentiments = [(chunk, sentiment_analysis(chunk)[0]) for chunk in chunks]
    return sentiments

def compute_weighted_sentiment(sentiment_results):
    total_positive_score = 0
    total_negative_score = 0
    total_neutral_score = 0
    total_length = 0

    for chunk, result in sentiment_results:
        length = len(chunk)
        total_length += length

        if result['label'] == "positive":
            total_positive_score += result['score'] * length
        elif result['label'] == "negative":
            total_negative_score += result['score'] * length
        else:
            total_neutral_score += result['score'] * length

    # Promedio ponderado
    weighted_positive_score = total_positive_score / total_length
    weighted_negative_score = total_negative_score / total_length
    weighted_neutral_score = total_neutral_score / total_length

    # Determinar el sentimiento predominante
    max_score = max(weighted_positive_score, weighted_negative_score, weighted_neutral_score)
    if max_score > 0.55:
        if max_score == weighted_positive_score:
            predominant_label = "positive"
        elif max_score == weighted_negative_score:
            predominant_label = "negative"
        else:
            predominant_label = "neutral"
    else:
        predominant_label = "neutral"

    return {
        "predominant_label": predominant_label,
        "weighted_positive_score": weighted_positive_score,
        "weighted_negative_score": weighted_negative_score,
        "weighted_neutral_score": weighted_neutral_score
    }

dataframe['positive_score'] = None
dataframe['negative_score'] = None
dataframe['neutral_score'] = None
dataframe['predominant_label'] = None

# Analizar y almacenar resultados en el DataFrame
for index, review in dataframe['review'].items():
    sentiment_results = analyze_long_review(review)
    overall_sentiment = compute_weighted_sentiment(sentiment_results)

    dataframe.at[index, 'positive_score'] = overall_sentiment['weighted_positive_score']
    dataframe.at[index, 'negative_score'] = overall_sentiment['weighted_negative_score']
    dataframe.at[index, 'neutral_score'] = overall_sentiment['weighted_neutral_score']
    dataframe.at[index, 'predominant_label'] = overall_sentiment['predominant_label']

print(dataframe.head())

   Unnamed: 0                                             review  score  \
0       21464  After waiting years for the sequel, this was s...      4   
1       21465  To make things simple, gameplay and graphics a...      4   
2       21466  Can't say it's a particularly enjoyable game, ...      4   
3       21467  Decent gameplay and fantastic visuals along wi...      4   
4       21468  Severely lacking in the story department to th...      4   

  positive_score negative_score neutral_score predominant_label  
0            0.0       0.955811           0.0          negative  
1            0.0       0.903573           0.0          negative  
2            0.0       0.626877           0.0          negative  
3            0.0       0.726832           0.0          negative  
4            0.0       0.883929           0.0          negative  


In [ ]:
with pd.ExcelWriter('sent_df_.xlsx') as writer:
    dataframe.to_excel(writer, sheet_name='dataframe_0' )

files.download('sent_df_.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>